Script en celda

En dos terminales ejecutar:

```bash
docker exec -it kafka bash
```
y luego en uno esto:

```bash
 ./opt/kafka/bin/kafka-console-consumer.sh \
  --bootstrap-server localhost:9092 \
  --topic mydata_prediction_response
```

y en otro:
```bash
/opt/kafka/bin/kafka-console-producer.sh   --broker-list kafka:9092   --topic mydata_prediction_request   --property "parse.key=false"   --property "key.separator=:"   --property "value.serializer=org.apache.kafka.common.serialization.StringSerializer"
```
y de ejemplo en el producer:
```bash
{"order_id": 1, "customer_id": "cust_123", "restaurant_id": "rest_456", "order_date_and_time": "2025-08-06T13:00:00", "delivery_date_and_time": "2025-08-06T13:45:00", "order_value": 1000, "delivery_fee": 100, "payment_method": "credit_card", "discounts_and_offers": "10% off", "commission_fee": 100, "payment_processing_fee": 20, "refunds/chargebacks": 0}
```

Pudes abrir mongo express y ver el resultado http://localhost:8081/db/agile_data_science/mydata_prediction_response

Para ejecutar el form:
```bash
docker exec -it agile bash
``` 
y dentro de agile:
```bash
python /home/jovyan/Food_delivery/Deploying_Predictive_Systems/web/MY_flask_api.py
``` 


In [1]:
# -*- coding: utf-8 -*-
# CELDA ÚNICA: Streaming Kafka -> Enriquecido -> Predicción -> Mongo + Kafka + Elasticsearch

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, LongType
from pyspark.sql.functions import from_json, col, hour, dayofweek, when, current_timestamp
from pyspark.ml import PipelineModel

print("🔧 Creando SparkSession...")
spark = (
    SparkSession.builder.appName("mydata-streaming-predict")
    .master("spark://agile:7077")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
print("✅ SparkSession creada")

# -------------------------
# 1) Cargar modelo
# -------------------------
print("🔧 Cargando PipelineModel...")
modelo = PipelineModel.load("./models/pipeline_model.bin")
print("✅ Modelo cargado")

# -------------------------
# 2) Esquema de entrada (incluye UUID)
# -------------------------
schema = StructType([
    StructField("UUID", StringType(), True),
    StructField("order_id", LongType(), True),
    StructField("customer_id", StringType(), True),
    StructField("restaurant_id", StringType(), True),
    StructField("order_date_and_time", TimestampType(), True),
    StructField("delivery_date_and_time", TimestampType(), True),
    StructField("order_value", DoubleType(), True),
    StructField("delivery_fee", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("discounts_and_offers", StringType(), True),
    StructField("commission_fee", DoubleType(), True),
    StructField("payment_processing_fee", DoubleType(), True),
    StructField("refunds/chargebacks", DoubleType(), True)
])

# -------------------------
# 3) Lectura desde Kafka
# -------------------------
print("🔌 Conectando a Kafka (topic: mydata_prediction_request)...")
raw_stream = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "mydata_prediction_request")
    .option("startingOffsets", "latest")   # cambia a "earliest" si quieres re-consumir desde el inicio
    .load()
)

json_df = (
    raw_stream
    .selectExpr("CAST(value AS STRING) AS json_data")
    .select(from_json("json_data", schema).alias("data"))
    .select("data.*")
)

# -------------------------
# 4) Enriquecido EXACTO como en el entrenamiento
# -------------------------
df_enriched = (
    json_df
    .withColumn("day_of_week", dayofweek("order_date_and_time"))
    .withColumn("hour_of_day", hour("order_date_and_time"))
    .withColumn("es_fin_de_semana", when(col("day_of_week").isin([1, 7]), 1).otherwise(0))
    .withColumn("es_hora_punta", when(
        (col("hour_of_day").between(13, 15)) | (col("hour_of_day").between(20, 22)), 1
    ).otherwise(0))
    .withColumn("has_discount", when(col("discounts_and_offers").isNotNull(), 1).otherwise(0))
    .withColumn("discount_value", when(col("has_discount") == 1, col("order_value") * 0.1).otherwise(0.0))
    .withColumn("refunded", when(col("refunds/chargebacks") > 0, 1).otherwise(0))
)

# -------------------------
# 5) Selección de columnas:
#    - las que el pipeline necesita
#    - + columnas de CONTEXTO que queremos conservar (pasan a través del modelo)
# -------------------------
required_numeric = [
    "order_value", "delivery_fee", "commission_fee", "payment_processing_fee",
    "refunds/chargebacks", "discount_value", "has_discount", "refunded",
    "day_of_week", "hour_of_day", "es_fin_de_semana", "es_hora_punta"
]
required_categorical = ["payment_method", "discounts_and_offers"]
context_cols = ["UUID", "order_id", "order_date_and_time", "payment_method", "discounts_and_offers"]
features_df = df_enriched.select(*(required_numeric + required_categorical + context_cols))

# -------------------------
# 6) Predicción
# -------------------------
predicciones = modelo.transform(features_df)

# Documento final (sin joins)
resultado_enriquecido = (
    predicciones
    .select(
        "UUID",
        "order_id",
        "prediction",
        "order_date_and_time",
        "payment_method",
        "discounts_and_offers",
        "day_of_week",
        "hour_of_day",
        "es_fin_de_semana",
        "es_hora_punta"
    )
    .withColumn("@ingest_ts", current_timestamp())
)

# -------------------------
# 7) foreachBatch: Mongo + Kafka + Elasticsearch (Bulk)
# -------------------------
def write_to_mongo_kafka_es(batch_df, epoch_id):
    import pymongo, json, urllib.request, urllib.error
    from uuid import uuid4

    client = None
    try:
        # Colectar microbatch (en dicts normales)
        rows = [r.asDict() for r in batch_df.collect()]
        print(f"🧱 Microbatch {epoch_id}: {len(rows)} docs")

        if not rows:
            return

        # --- Prepara versión SANEADA para ES (NUNCA con _id)
        es_rows = []
        for d in rows:
            d_es = dict(d)        # copia
            d_es.pop("_id", None) # por si viniera de otra etapa
            es_rows.append(d_es)

        # --- Construye y envía BULK NDJSON a ES ANTES de tocar Mongo
        ndjson_lines = []
        for d in es_rows:
            es_id = d.get("UUID") or str(uuid4())  # idempotencia
            ndjson_lines.append(json.dumps({"index": {"_index": "mydata_prediction_response", "_id": es_id}}))
            ndjson_lines.append(json.dumps(d, default=str))
        payload = ("\n".join(ndjson_lines) + "\n").encode("utf-8")

        try:
            req = urllib.request.Request(
                "http://elastic:9200/_bulk",
                data=payload,
                headers={"Content-Type": "application/x-ndjson"},
                method="POST",
            )
            with urllib.request.urlopen(req, timeout=15) as resp:
                body = resp.read().decode("utf-8", errors="replace")
                print("📦 ES bulk resp:", body[:500], "...")
        except urllib.error.HTTPError as he:
            body = he.read().decode("utf-8", errors="replace")
            print(f"❌ ES HTTPError {he.code}: {body[:800]}")

        # --- Kafka (respuesta): usa el batch_df tal cual
        (batch_df
         .selectExpr("UUID as key", "to_json(struct(*)) as value")
         .write
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka:9092")
         .option("topic", "mydata_prediction_response")
         .save())

        # --- Mongo: inserta COPIAS para que PyMongo pueda inyectar _id sin contaminar nada
        client = pymongo.MongoClient("mongo")
        db = client["agile_data_science"]
        out = db["mydata_prediction_response"]
        out.create_index("UUID", unique=False)

        rows_for_mongo = [dict(d) for d in es_rows]  # copia independiente
        out.insert_many(rows_for_mongo)

    except Exception as e:
        print("❌ Error en foreachBatch:", e)
        try:
            if client is None:
                client = pymongo.MongoClient("mongo")
            db = client["agile_data_science"]
            db["mydata_prediction_errors"].insert_one({
                "epoch_id": int(epoch_id),
                "error": str(e),
                "note": "Fallo en foreachBatch",
            })
        except Exception as e2:
            print("❌ Error registrando error en Mongo:", e2)
    finally:
        try:
            client and client.close()
        except:
            pass

# -------------------------
# 8) Lanzar el stream
# -------------------------
print("🚀 Iniciando streaming (Mongo + Kafka + Elasticsearch)...")
query = (resultado_enriquecido.writeStream
         .outputMode("append")
         .option("checkpointLocation", "/tmp/checkpoints-foreachbatch-es-v6")  # cambia si reinicias
         .foreachBatch(write_to_mongo_kafka_es)
         .start())

print("⏳ Esperando microbatches...")
query.awaitTermination()

🔧 Creando SparkSession...


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4bd93764-a28e-4247-927f-6ed91910a4f2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central

✅ SparkSession creada
🔧 Cargando PipelineModel...


✅ Modelo cargado
🔌 Conectando a Kafka (topic: mydata_prediction_request)...
🚀 Iniciando streaming (Mongo + Kafka + Elasticsearch)...
⏳ Esperando microbatches...
🧱 Microbatch 0: 0 docs


🧱 Microbatch 1: 1 docs
📦 ES bulk resp: {"errors":true,"took":0,"ingest_took":0,"items":[{"index":{"_index":"mydata_prediction_response","_id":"E-aKqZgBZOyNk9YBcnsU","status":400,"error":{"type":"document_parsing_exception","reason":"[1:285] failed to parse field [@ingest_ts] of type [date] in document with id 'E-aKqZgBZOyNk9YBcnsU'. Preview of field's value: '2025-08-14 17:04:32.973000'","caused_by":{"type":"illegal_argument_exception","reason":"failed to parse date field [2025-08-14 17:04:32.973000] with format [strict_date_optional ...


ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.2-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=64>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.2-src.zip/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.2-src.zip/py4j/clientserver.py", line 475, in send_command
    answer

Py4JError: An error occurred while calling o552.awaitTermination